In [2]:
import cv2
import dlib
import numpy as np
import os
import pandas as pd # Import pandas for CSV handling
# Removed 'math' as rotation logic is no longer needed

# Load detector and predictor
detector = dlib.get_frontal_face_detector()
# Adjusted path to be relative to the project folder, assuming 'models' is a direct child
# of the main project folder.
predictor = dlib.shape_predictor(
    "models/shape_predictor_68_face_landmarks.dat/shape_predictor_68_face_landmarks.dat"
)

# Define the root directories for input and output
root_input_dir = "archive_3"
root_output_dir = "processed_faces_newFinal7_2" # This is your main output directory

# Define the scaling factor for upscaling images
scale_factor = 12

# Create the root output directory if it doesn't exist
os.makedirs(root_output_dir, exist_ok=True)

# List of emotion folders
emotion_folders = ["anger", "contempt", "disgust", "happy", "sadness", "fear", "surprise"]

# List to store all landmark data for CSV export (will be expanded later for fitted parameters)
all_landmarks_data = []

print("Starting facial landmark detection and processing for ALL photos...")
print("Output images will be split into '_with_photo', '_without_photo', and '_shapes_drawn' folders.")
print("A comprehensive CSV file with landmark coordinates will also be generated.")

# --- Helper Function for Drawing Shapes ---
def draw_facial_shapes(image, landmarks):
    """
    Draws different geometric shapes on a given image based on facial landmarks.
    Args:
        image (numpy.ndarray): The image on which to draw.
        landmarks (dlib.full_object_detection): Dlib's landmark object.
    Returns:
        numpy.ndarray: The image with shapes drawn.
    """
    drawn_image = image.copy() # Draw on a copy to avoid modifying original

    # Define colors for different shapes (BGR format)
    color_eyebrows = (0, 255, 0)   # Green
    color_eyes = (255, 0, 0)       # Blue
    color_nose_bridge = (0, 255, 255) # Yellow
    color_nose_tip = (255, 255, 0) # Cyan
    color_lips = (0, 165, 255)     # Orange
    color_cheeks = (128, 0, 128)   # Purple (new position for cheeks)
    thickness = 2 # Line thickness

    # --- Eyebrows (Approximated by polylines) ---
    # Left Eyebrow: 17-21
    for i in range(17, 21): # Loop up to 21 to connect 20 and 21
        pt1 = (landmarks.part(i).x, landmarks.part(i).y)
        pt2 = (landmarks.part(i + 1).x, landmarks.part(i + 1).y)
        cv2.line(drawn_image, pt1, pt2, color_eyebrows, thickness)

    # Right Eyebrow: 22-26
    for i in range(22, 26): # Loop up to 26 to connect 25 and 26
        pt1 = (landmarks.part(i).x, landmarks.part(i).y)
        pt2 = (landmarks.part(i + 1).x, landmarks.part(i + 1).y)
        cv2.line(drawn_image, pt1, pt2, color_eyebrows, thickness)

    # --- Eyes (Ellipses) ---
    # Left Eye: 36-41
    left_eye_points = np.array([(landmarks.part(n).x, landmarks.part(n).y) for n in range(36, 42)])
    if len(left_eye_points) >= 5: # cv2.fitEllipse requires at least 5 points
        (center_le, axes_le, angle_le) = cv2.fitEllipse(left_eye_points)
        if axes_le[0] > 0 and axes_le[1] > 0:
            cv2.ellipse(drawn_image, (int(center_le[0]), int(center_le[1])),
                        (int(axes_le[0] / 2), int(axes_le[1] / 2)),
                        angle_le, 0, 360, color_eyes, thickness)

    # Right Eye: 42-47
    right_eye_points = np.array([(landmarks.part(n).x, landmarks.part(n).y) for n in range(42, 48)])
    if len(right_eye_points) >= 5:
        (center_re, axes_re, angle_re) = cv2.fitEllipse(right_eye_points)
        if axes_re[0] > 0 and axes_re[1] > 0:
            cv2.ellipse(drawn_image, (int(center_re[0]), int(center_re[1])),
                        (int(axes_re[0] / 2), int(axes_re[1] / 2)),
                        angle_re, 0, 360, color_eyes, thickness)

    # --- Nose ---
    # Nose Bridge (Trapezium): Using landmarks 27, 28, 29, 30
    # nose_bridge_pts = np.array([
    #     (landmarks.part(27).x, landmarks.part(27).y),
    #     (landmarks.part(28).x, landmarks.part(28).y),
    #     (landmarks.part(29).x, landmarks.part(29).y),
    #     (landmarks.part(30).x, landmarks.part(30).y)
    # ], np.int32).reshape((-1, 1, 2))
    # cv2.polylines(drawn_image, [nose_bridge_pts], True, color_nose_bridge, thickness)
    
    
    # --- Nose ---
    # Nose Bridge: Draw a thick vertical line between landmark 27 and 30
    pt_top = (landmarks.part(27).x, landmarks.part(27).y)
    pt_bottom = (landmarks.part(30).x, landmarks.part(30).y)
    cv2.line(drawn_image, pt_top, pt_bottom, color_nose_bridge, thickness=4)


    # Nose Tip (Triangle): Using landmarks 30, 31, 35
    nose_tip_pts = np.array([
        (landmarks.part(30).x, landmarks.part(30).y),
        (landmarks.part(31).x, landmarks.part(31).y),
        (landmarks.part(35).x, landmarks.part(35).y)
    ], np.int32).reshape((-1, 1, 2))
    cv2.polylines(drawn_image, [nose_tip_pts], True, color_nose_tip, thickness)

    # --- Lips (Outer Lip Ellipse) ---
    # Outer Lip: 48-59
    outer_lip_points = np.array([(landmarks.part(n).x, landmarks.part(n).y) for n in range(48, 60)])
    if len(outer_lip_points) >= 5:
        (center_ol, axes_ol, angle_ol) = cv2.fitEllipse(outer_lip_points)
        if axes_ol[0] > 0 and axes_ol[1] > 0:
            cv2.ellipse(drawn_image, (int(center_ol[0]), int(center_ol[1])),
                        (int(axes_ol[0] / 2), int(axes_ol[1] / 2)),
                        angle_ol, 0, 360, color_lips, thickness)

    # --- Cheeks (Approximated by Broader Triangles) ---
    # Left Cheek: Using outer eye corner (36), outer jawline (2), left mouth corner (48)
    left_cheek_pts = np.array([
        (landmarks.part(36).x, landmarks.part(36).y), # Left outer eye corner
        (landmarks.part(2).x, landmarks.part(2).y),   # Left jawline (near chin)
        (landmarks.part(48).x, landmarks.part(48).y)  # Left mouth corner
    ], np.int32).reshape((-1, 1, 2))
    cv2.polylines(drawn_image, [left_cheek_pts], True, color_cheeks, thickness)

    # Right Cheek: Using outer eye corner (45), outer jawline (14), right mouth corner (54)
    right_cheek_pts = np.array([
        (landmarks.part(45).x, landmarks.part(45).y), # Right outer eye corner
        (landmarks.part(14).x, landmarks.part(14).y), # Right jawline (near chin)
        (landmarks.part(54).x, landmarks.part(54).y)  # Right mouth corner
    ], np.int32).reshape((-1, 1, 2))
    cv2.polylines(drawn_image, [right_cheek_pts], True, color_cheeks, thickness)

    return drawn_image

# --- Main Processing Loop ---
for emotion in emotion_folders:
    input_emotion_path = os.path.join(root_input_dir, emotion)

    # Define the new output subfolders for each emotion
    output_with_photo_path = os.path.join(root_output_dir, emotion, f"{emotion}_with_photo")
    output_without_photo_path = os.path.join(root_output_dir, emotion, f"{emotion}_without_photo")
    output_shapes_drawn_path = os.path.join(root_output_dir, emotion, f"{emotion}_shapes_drawn") # New folder for shapes

    # Create these new output subdirectories if they don't exist
    os.makedirs(output_with_photo_path, exist_ok=True)
    os.makedirs(output_without_photo_path, exist_ok=True)
    os.makedirs(output_shapes_drawn_path, exist_ok=True) # Create shapes folder

    print(f"\nProcessing emotion: {emotion}...")

    for filename in os.listdir(input_emotion_path):
        if filename.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".tiff")):
            image_path = os.path.join(input_emotion_path, filename)
            image = cv2.imread(image_path)

            if image is None:
                print(f"Warning: Could not read image {image_path}. Skipping.")
                continue

            print(f"  Processing file: {filename}")

            upscaled_image = cv2.resize(image, (0, 0), fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_CUBIC)
            gray = cv2.cvtColor(upscaled_image, cv2.COLOR_BGR2GRAY)

            faces = detector(gray)

            if len(faces) > 0:
                face = faces[0]
                landmarks = predictor(gray, face)

                # --- Collect landmark data for CSV ---
                landmark_row = {'emotion': emotion, 'filename': filename}
                for n in range(68):
                    x = landmarks.part(n).x
                    y = landmarks.part(n).y
                    landmark_row[f'landmark_{n}_x'] = x
                    landmark_row[f'landmark_{n}_y'] = y
                all_landmarks_data.append(landmark_row)
                # --- End of CSV data collection ---

                # --- Prepare images for all output types ---
                drawn_image_with_photo = upscaled_image.copy()
                drawn_image_without_photo = np.zeros(upscaled_image.shape, dtype=np.uint8)
                drawn_image_shapes_only = np.zeros(upscaled_image.shape, dtype=np.uint8) # New blank image for shapes

                # Get position of landmark 31 (index 30 in 0-based indexing) for quadrant lines
                nose_x = landmarks.part(30).x
                nose_y = landmarks.part(30).y

                # Draw quadrant lines centered at landmark 31 (only on image WITH photo)
                img_h, img_w = drawn_image_with_photo.shape[:2]
                cv2.line(drawn_image_with_photo, (nose_x, 0), (nose_x, img_h), (0, 255, 0), 1) # Green line
                cv2.line(drawn_image_with_photo, (0, nose_y), (img_w, nose_y), (0, 255, 0), 1) # Green line

                drawn_index = 1
                seen = set()

                for n in range(68):
                    x = landmarks.part(n).x
                    y = landmarks.part(n).y

                    if (x, y) in seen:
                        continue
                    seen.add((x, y))

                    circle_radius = 2
                    fill_type = -1

                    # Offset for label (re-using your existing logic for good labels)
                    offset_x, offset_y = -6, 10
                    if n in [37, 38, 39, 43, 44, 45]: # Upper eye points
                        offset_y = -10
                    elif n == 49: # Left mouth corner
                        offset_x = -20
                        offset_y = 6
                    elif n == 50: # Top lip center
                        offset_x = 0
                        offset_y = -8
                    elif n == 65: # Chin left
                        offset_x = -14
                        offset_y = 10
                    elif 48 <= n <= 54: # Upper lip
                        offset_y = -8
                    elif 55 <= n <= 59: # Lower lip center to right
                        offset_y = 10
                    elif 60 <= n <= 64: # Inner mouth top
                        offset_y = -8
                    elif 65 <= n <= 67: # Chin right
                        offset_y = 10

                    # Draw on the image WITH photo (original background)
                    text_color_with_photo = (0, 0, 0) # Black text
                    cv2.circle(drawn_image_with_photo, (x, y), circle_radius, (0, 0, 255), fill_type) # Red circles
                    cv2.putText(drawn_image_with_photo, str(drawn_index), (x + offset_x, y + offset_y),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.35, text_color_with_photo, 1, cv2.LINE_AA)

                    # Draw on the image WITHOUT photo (black background)
                    text_color_without_photo = (255, 255, 255) # White text/circles
                    cv2.circle(drawn_image_without_photo, (x, y), circle_radius, text_color_without_photo, fill_type)
                    cv2.putText(drawn_image_without_photo, str(drawn_index), (x + offset_x, y + offset_y),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.35, text_color_without_photo, 1, cv2.LINE_AA)

                    drawn_index += 1

                # --- Draw the specific shapes on the dedicated 'shapes_only' image ---
                drawn_image_shapes_only = draw_facial_shapes(drawn_image_shapes_only, landmarks)


                # Save the processed images to their respective folders
                output_image_path_with_photo = os.path.join(output_with_photo_path, filename)
                cv2.imwrite(output_image_path_with_photo, drawn_image_with_photo)

                output_image_path_without_photo = os.path.join(output_without_photo_path, filename)
                cv2.imwrite(output_image_path_without_photo, drawn_image_without_photo)

                output_image_path_shapes_drawn = os.path.join(output_shapes_drawn_path, filename) # New save path
                cv2.imwrite(output_image_path_shapes_drawn, drawn_image_shapes_only) # Save the shapes-only image

            else:
                print(f"  No face detected in {filename} from {emotion} folder.")

print("\nAll emotion folders and photos processed successfully!")

# --- Save all collected landmark data to a single CSV file ---
if all_landmarks_data:
    df = pd.DataFrame(all_landmarks_data)
    csv_output_path = os.path.join(root_output_dir, "facial_landmarks_data.csv")
    df.to_csv(csv_output_path, index=False)
    print(f"All facial landmark coordinates saved to: {csv_output_path}")
else:
    print("No facial landmark data was collected (no faces detected or no images processed).")

Starting facial landmark detection and processing for ALL photos...
Output images will be split into '_with_photo', '_without_photo', and '_shapes_drawn' folders.
A comprehensive CSV file with landmark coordinates will also be generated.

Processing emotion: anger...
  Processing file: S010_004_00000017.png
  Processing file: S010_004_00000018.png
  Processing file: S010_004_00000019.png
  Processing file: S011_004_00000019.png
  Processing file: S011_004_00000020.png
  Processing file: S011_004_00000021.png
  Processing file: S014_003_00000028.png
  Processing file: S014_003_00000029.png
  Processing file: S014_003_00000030.png
  Processing file: S022_005_00000030.png
  Processing file: S022_005_00000031.png
  Processing file: S022_005_00000032.png
  Processing file: S026_003_00000013.png
  Processing file: S026_003_00000014.png
  Processing file: S026_003_00000015.png
  Processing file: S028_001_00000022.png
  Processing file: S028_001_00000023.png
  Processing file: S028_001_0000002